In [98]:
import numpy as np
from scipy.optimize import dual_annealing
import pulp
import pandas as pd
import os
import random
from wmape import WMAPE
from order_generator import OrderGenerator
from order_merger import OrderMerger
import math

In [99]:
random.seed(42)

In [101]:
# import allocations for lead day 5 and lead day 0 and calculate wmape values

allocation_dir = 'allocations'
merged_dir = 'merged'
lead_day = 5
soft_col = f'lead_{lead_day}'
hard_col = 'lead_0'

allocation_0 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_0.csv')
allocation_5 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_5.csv')

merged_5_0 = pd.read_csv(f'{merged_dir}/merged_lead_day_{lead_day}_0.csv')

_, wmape_site = WMAPE.calculate_wmape_site(merged_5_0, soft_col, hard_col)
_, wmape_global = WMAPE.calculate_wmape_global(merged_5_0, soft_col, hard_col)

print(f'wmape_global: {wmape_global}')
print(f'wmape_site: {wmape_site}')

wmape_global: 0.21075893420886477
wmape_site: 0.42063391842404346


In [102]:
def objective_function(current_allocation, orders_df, actual_orders_df):
    # Apply the current allocation to the orders dataframe
    orders_df['site'] = current_allocation

    # Aggregate the orders for lead day 5
    aggregated_5_df = OrderMerger.aggregate_items(orders_df, 5)
    aggregated_0_df = OrderMerger.aggregate_items(actual_orders_df, 0)

    # Merge the aggregated dataframes
    merged_df = OrderMerger.merge_allocation(aggregated_5_df, aggregated_0_df, 5, 0)

    # Calculate WMAPE
    _, wmape_site = WMAPE.calculate_wmape_site(merged_df, 'lead_5', 'lead_0')
    return wmape_site

In [103]:
def cooling_schedule(initial_temp, alpha, step):
    return initial_temp * (alpha ** step)


def generate_neighbor(current_allocation):
    new_allocation = current_allocation.copy()
    
    # Randomly swap two sites in the allocation
    idx1, idx2 = random.sample(range(len(new_allocation)), 2)
    new_allocation[idx1], new_allocation[idx2] = new_allocation[idx2], new_allocation[idx1]
    
    return new_allocation


def acceptance_probability(current_energy, new_energy, temperature):
    if new_energy < current_energy:
        return 1.0
    else:
        return math.exp((current_energy - new_energy) / temperature)


def simulated_annealing(orders_df, actual_orders_df, initial_temp, final_temp, alpha, max_iter):
    current_allocation = orders_df['site'].values
    best_allocation = current_allocation.copy()
    current_energy = objective_function(current_allocation, orders_df, actual_orders_df)
    best_energy = current_energy

    temp = initial_temp
    step = 0

    while temp > final_temp and step < max_iter:
        new_allocation = generate_neighbor(current_allocation)
        new_energy = objective_function(new_allocation, orders_df, actual_orders_df)

        if acceptance_probability(current_energy, new_energy, temp) > random.random():
            current_allocation = new_allocation
            current_energy = new_energy

            if current_energy < best_energy:
                best_allocation = current_allocation
                best_energy = current_energy

        temp = cooling_schedule(initial_temp, alpha, step)
        step += 1

    return best_allocation, best_energy

In [114]:
# Define the parameters for simulated annealing
initial_temp = 5000
final_temp = 1
alpha = 0.95
max_iter = 100000

# Run the simulated annealing optimization
best_allocation, best_energy = simulated_annealing(allocation_5.copy(), allocation_0.copy(), initial_temp, final_temp, alpha, max_iter)

# Apply the best allocation to the orders dataframe
new_allocation_5 = allocation_5.copy()
new_allocation_5['site'] = best_allocation

print("Best WMAPE:", best_energy)
print("Best Allocation:", best_allocation)

Best WMAPE: 0.41861346129561816
Best Allocation: ['F11' 'F11' 'F11' ... 'F20' 'F20' 'F20']
